### Note: This is only applicable to Rhino. The principles of parallel computing are general and can be used to accelerate code on any system, but the implementations here are designed to interact with the scheduler used on the Computational Memory Lab cluster.

## Parallel Computing with Dask

<img src="https://hpc.llnl.gov/sites/default/files/styles/with_sidebar_1_up/public/nodesNetwork.gif?itok=TBqDQmx0">

A node is a like a computer within a much bigger computer!

**CMLDask wrapper**:
* See https://github.com/pennmem/cmldask for wrapper package created for convenience using Rhino + typical parallel setup for our analyses
* Check out [Dask](https://docs.dask.org/en/stable/) and [Dask distributed](https://distributed.dask.org/en/stable/) for more documentation about the underlying implementation. In particular, we're using the [Futures API](https://docs.dask.org/en/stable/futures.html?highlight=futures) for managing parallel tasks and [dask-jobqueue](http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SGECluster.html#dask_jobqueue.SGECluster) to connect with the Sun Grid Engine (SGE) scheduler on rhino. 
* Note that this package is intended for convenience and will not accomodate very specific computational needs that demand complex parallel architectures. You will need to use dask directly and create your own `Cluster()` and `Client()` instances that are more closely tailored to your needs.

### Basic usage works as follows:

In [1]:
import cmldask.CMLDask as da

/usr/global/Anaconda/2019-10/envs/cml37/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
import cmldask
from dask.distributed import wait, as_completed, progress

def squared(x):
    return x**2

client = da.new_dask_client("test_dask", "1GB")

Unique port for jrudoler is 51360
{'dashboard_address': ':51360'}
To view the dashboard, run: 
`ssh -fN jrudoler@rhino2.psych.upenn.edu -L 8000:192.168.86.141:51360` in your local computer's terminal (NOT rhino) 
and then navigate to localhost:8000 in your browser


**Note that you need to explicitly shut down this client (`client.shutdown()`) or shut down your jupyter kernel before running this cell again, or you might leave workers "stranded" without access to them (since that access is provided by the Client you would overwrite)**

You have options when setting up your Dask cluster/client:

In [6]:
da.new_dask_client?

Signature: da.new_dask_client(job_name, memory_per_job, max_n_jobs=100, threads_per_job=1, processes_per_job=1, adapt=True, queue='RAM.q', walltime='1500000', local_directory=None, log_directory=None, scheduler_options={'dashboard_address': ':51360'}, **kwargs)
Docstring:
Returns a new dask client instance with associated dashboard for
monitoring jobs. The default method assumes a very basic use case - that
is, embarassingly parallel tasks - in which the user simply wants to speed
up independent computations by running them in parallel. There is just one
single-threaded process per job. The thread/process parameters are
configurable, but will lead to more complicated parallel executions
which might be hard to track. Workers might share memory instead of
being independent.
Leave the defaults unless you have some specific reason to believe another
configuration will help with your use case.

Parameters
----------
job_name : gives every job this client runs a shared name. You can
    subs

## Parallel computing tips

* Jobs are still subject to memory limitations, so you may need to **break up large processes into smaller chunks.** For example, each job could correspond to analyzing one session, instead of one subject. 
* It is often useful to save the output of each job in a dedicated directory, and sometimes useful to save intermediate values to aid in debugging or later nonparallel analyses. The Python "os" library can be helpful here. 
* **Be respectful!** There are only so many cores available to the entire Kahana lab and our collaborators across the country. 
* **Limit typical jobs to 100 cores or less**. Heavy usage means fewer resources for other users to use, and due to shared disk resources might actually slow down all jobs overall. Please ask for permission before using more.
* You can always use the '**qdel**' command in Terminal, followed by your job number, to kill any of your old jobs that may be wasting rhino's resources. 
* Use the '**qstat**' command in Terminal to see cluster usage information.
* Each rhino2 node has ~128 GB of memory and ~40 cores. 


### Simple Dask Futures Usage

In [4]:
import numpy as np
from time import sleep

def add(a, b):
    sleep(np.random.randint(0, 10))
    return a + b

def error_add(a, b):
    sleep(np.random.randint(0, 10))
    if a % 2:
        raise ValueError
    return a + b

futures = client.map(add, range(40), range(40))
progress(futures)

VBox()

### Handling Errors

Now with errors

In [7]:
futures = client.map(error_add, range(40), range(40))

Wait for all jobs to finish, check for errors

In [9]:
wait(futures)
errors = da.get_exceptions(futures, range(40))
errors

,param,exception,traceback_obj
index,,,
1,1,ValueError(),<traceback object at 0x2b004e8a38c0>
3,3,ValueError(),<traceback object at 0x2b004e99af00>
5,5,ValueError(),<traceback object at 0x2b004e9bbb90>
7,7,ValueError(),<traceback object at 0x2b004ea60dc0>
9,9,ValueError(),<traceback object at 0x2b004ea0bc80>
11,11,ValueError(),<traceback object at 0x2b004e8c9d70>
13,13,ValueError(),<traceback object at 0x2b004ea1a6e0>
15,15,ValueError(),<traceback object at 0x2b004e98bb90>
17,17,ValueError(),<traceback object at 0x2b004e9bdd20>


Pick out the index where you want to view the traceback message

In [11]:
da.print_traceback(errors, 5)

  File "<ipython-input-4-492535f77c2d>", line 11, in error_add
    raise ValueError


Notice gathering these doesn't work because there are errors

In [12]:
client.gather(futures)

ValueError: 

Instead, let's filter for successful ones

In [14]:
good_futures = da.filter_futures(futures)
client.gather(good_futures)

[0, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76]

## IMPORTANT: Shutdown your client (or restart your kernel, which will do so automatically)

In [15]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


Happy parallel computing!

**Exercise: Write a parallel function that returns the number of (bipolar) electrodes for every subject in the RAM example dataset. Run with 5 jobs and 1 core per job. This will require you to integrate the data loading procedures you learned earlier with the dask framework exhibited using the toy examples above**